<a href="https://colab.research.google.com/github/0-Bhavin/Job-Finder-AI-Agent/blob/main/GENAIAsiignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os

project_root = "/content/drive/My Drive/job_search_ai"
creds_dir = os.path.join(project_root, "creds")
resume_dir = os.path.join(project_root, "resume")

os.makedirs(creds_dir, exist_ok=True)
os.makedirs(resume_dir, exist_ok=True)

print("✅ Project directories created!")

✅ Project directories created!


In [ ]:
!pip install streamlit openai google-auth google-auth-oauthlib google-api-python-client requests python-docx PyPDF2 google-generativeai


In [ ]:
import streamlit as st
import requests
import google.generativeai as genai
from PyPDF2 import PdfReader
from datetime import datetime, timedelta
import json
import os

GENAI_API_KEY = "  "
AFFINDA_API_KEY = "aff_131c9ac44d037d14bae2616babff27c9fb874947"
REMOTIVE_URL = "https://remotive.com/api/remote-jobs"

# Setup Gemini
genai.configure(api_key=GENAI_API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash-lite-001")

#  Function: Resume Parser
def parse_resume_with_affinda(resume_file):
    headers = {
        "Authorization": f"Bearer {AFFINDA_API_KEY}",
    }
    files = {"file": resume_file}
    response = requests.post("https://api.affinda.com/v2/resumes", headers=headers, files=files)
    if response.status_code == 201:
        data = response.json()
        return data["data"]["professionalSummary"], data["data"]["skills"]
    else:
        return None, None

#  Search Jobs
def search_jobs(keyword, location):
    response = requests.get(REMOTIVE_URL)
    if response.status_code == 200:
        all_jobs = response.json()["jobs"]
        results = []
        for job in all_jobs:
            if keyword.lower() in job["title"].lower() and location.lower() in job["candidate_required_location"].lower():
                results.append(job)
        return results
    return []

#  Function: Suggest Interview Prep
def interview_prep_with_gemini(job_title):
    prompt = f"Give me a list of 5 common interview questions and sample answers for the position: {job_title}"
    response = model.generate_content(prompt)
    return response.text

# Function: Add to Google Calendar
def schedule_event(summary, start_time, duration=1):
    from google.oauth2 import service_account
    from googleapiclient.discovery import build

    creds = service_account.Credentials.from_service_account_file(
        'creds/bold-sorter-462413-e0-6cd62332c6d2.json.json',
        scopes=['https://www.googleapis.com/auth/calendar']
    )
    service = build('calendar', 'v3', credentials=creds)

    event = {
        'summary': summary,
        'start': {'dateTime': start_time.isoformat(), 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': (start_time + timedelta(hours=duration)).isoformat(), 'timeZone': 'Asia/Kolkata'},
    }

    created_event = service.events().insert(calendarId='primary', body=event).execute()
    return created_event.get('htmlLink')


st.title("🤖 Job Search AI Agent")

st.sidebar.header("1️⃣ Upload Resume")
resume_file = st.sidebar.file_uploader("Upload your resume (PDF/DOCX)", type=["pdf", "docx"])
resume_summary, resume_skills = None, None

if resume_file:
    with st.spinner("Parsing resume..."):
        resume_summary, resume_skills = parse_resume_with_affinda(resume_file)
    if resume_summary:
        st.sidebar.success("Resume parsed successfully!")
    else:
        st.sidebar.error("Parsing failed!")

st.sidebar.header("2️⃣ Job Search Filter")
keyword = st.sidebar.text_input("Job Title", value="Software Engineer")
location = st.sidebar.text_input("Location", value="India")
search = st.sidebar.button("Search Jobs")

if search:
    with st.spinner("Searching jobs..."):
        jobs = search_jobs(keyword, location)
        if jobs:
            st.subheader("🔍 Matching Jobs")
            for job in jobs[:5]:
                st.markdown(f"### {job['title']} at {job['company_name']}")
                st.markdown(f"📍 Location: {job['candidate_required_location']}")
                st.markdown(f"🔗 [Job Link]({job['url']})")
                st.markdown("---")
        else:
            st.warning("No jobs found.")

if st.button("💬 Gemini Interview Prep"):
    if keyword:
        with st.spinner("Preparing interview questions..."):
            content = interview_prep_with_gemini(keyword)
            st.markdown("### 🤖 Gemini Interview Tips")
            st.markdown(content)
    else:
        st.warning("Enter job title first.")

st.sidebar.header("3️⃣ Schedule Prep Session")
schedule_btn = st.sidebar.button("Schedule Interview Prep")
if schedule_btn:
    prep_time = datetime.now() + timedelta(days=1, hours=2)
    link = schedule_event("Interview Prep Session", prep_time)
    st.success(f"✅ Interview prep session scheduled!")
    st.markdown(f"[📅 View in Google Calendar]({link})")



2025-06-09 14:25:25.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-09 14:25:25.875 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

# LOcal Tunneing

In [ ]:
!pip install streamlit localtunnel


ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [ ]:
code = """
import streamlit as st
import requests
import google.generativeai as genai
from PyPDF2 import PdfReader
from datetime import datetime, timedelta
import json
import os

# =============== CONFIG =================
GENAI_API_KEY = "your_gemini_api_key"
AFFINDA_API_KEY = "your_affinda_api_key"
REMOTIVE_URL = "https://remotive.com/api/remote-jobs"

# Setup Gemini
genai.configure(api_key=GENAI_API_KEY)
model = genai.GenerativeModel("gemini-pro")

# ========== Function: Resume Parser ==========
def parse_resume_with_affinda(resume_file):
    headers = {
        "Authorization": f"Bearer {AFFINDA_API_KEY}",
    }
    files = {"file": resume_file}
    response = requests.post("https://api.affinda.com/v2/resumes", headers=headers, files=files)
    if response.status_code == 201:
        data = response.json()
        return data["data"]["professionalSummary"], data["data"]["skills"]
    else:
        return None, None

# ========== Function: Search Jobs ==========
def search_jobs(keyword, location):
    response = requests.get(REMOTIVE_URL)
    if response.status_code == 200:
        all_jobs = response.json()["jobs"]
        results = []
        for job in all_jobs:
            if keyword.lower() in job["title"].lower() and location.lower() in job["candidate_required_location"].lower():
                results.append(job)
        return results
    return []

# ========== Function: Suggest Interview Prep ==========
def interview_prep_with_gemini(job_title):
    prompt = f"Give me a list of 5 common interview questions and sample answers for the position: {job_title}"
    response = model.generate_content(prompt)
    return response.text

# ========== Function: Add to Google Calendar ==========
def schedule_event(summary, start_time, duration=1):
    from google.oauth2 import service_account
    from googleapiclient.discovery import build

    creds = service_account.Credentials.from_service_account_file(
        'creds/client_secret.json',
        scopes=['https://www.googleapis.com/auth/calendar']
    )
    service = build('calendar', 'v3', credentials=creds)

    event = {
        'summary': summary,
        'start': {'dateTime': start_time.isoformat(), 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': (start_time + timedelta(hours=duration)).isoformat(), 'timeZone': 'Asia/Kolkata'},
    }

    created_event = service.events().insert(calendarId='primary', body=event).execute()
    return created_event.get('htmlLink')

# ========== Streamlit App ==========
st.title("🤖 Job Search AI Agent")

st.sidebar.header("1️⃣ Upload Resume")
resume_file = st.sidebar.file_uploader("Upload your resume (PDF/DOCX)", type=["pdf", "docx"])
resume_summary, resume_skills = None, None

if resume_file:
    with st.spinner("Parsing resume..."):
        resume_summary, resume_skills = parse_resume_with_affinda(resume_file)
    if resume_summary:
        st.sidebar.success("Resume parsed successfully!")
    else:
        st.sidebar.error("Parsing failed!")

st.sidebar.header("2️⃣ Job Search Filter")
keyword = st.sidebar.text_input("Job Title", value="Software Engineer")
location = st.sidebar.text_input("Location", value="India")
search = st.sidebar.button("Search Jobs")

if search:
    with st.spinner("Searching jobs..."):
        jobs = search_jobs(keyword, location)
        if jobs:
            st.subheader("🔍 Matching Jobs")
            for job in jobs[:5]:
                st.markdown(f"### {job['title']} at {job['company_name']}")
                st.markdown(f"📍 Location: {job['candidate_required_location']}")
                st.markdown(f"🔗 [Job Link]({job['url']})")
                st.markdown("---")
        else:
            st.warning("No jobs found.")

if st.button("💬 Gemini Interview Prep"):
    if keyword:
        with st.spinner("Preparing interview questions..."):
            content = interview_prep_with_gemini(keyword)
            st.markdown("### 🤖 Gemini Interview Tips")
            st.markdown(content)
    else:
        st.warning("Enter job title first.")

st.sidebar.header("3️⃣ Schedule Prep Session")
schedule_btn = st.sidebar.button("Schedule Interview Prep")
if schedule_btn:
    prep_time = datetime.now() + timedelta(days=1, hours=2)
    link = schedule_event("Interview Prep Session", prep_time)
    st.success(f"✅ Interview prep session scheduled!")
    st.markdown(f"[📅 View in Google Calendar]({link})")

"""

with open("app.py", "w") as f:
    f.write(code)


In [ ]:
!streamlit run app.py &> /content/log.txt &


In [ ]:
!npx --yes localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://curly-corners-reply.loca.lt
^C


# NGROK

In [ ]:
!pip install streamlit pyngrok


In [ ]:
!streamlit run app.py &>/content/log.txt &


In [ ]:
!ngrok config add-authtoken 2vci9ZhdwisNytZrEaAl4o37gEM_3LAV1D2CWrEHCmS1mqPQk


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Corrected: use "http" scheme and bind to port 8501
public_url = ngrok.connect(8501, "http")
print(f"🔗 Public URL: {public_url}")


🔗 Public URL: NgrokTunnel: "https://33ff-34-135-202-150.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving bold-sorter-462413-e0-6cd62332c6d2.json to bold-sorter-462413-e0-6cd62332c6d2.json


In [ ]:
import os
print(os.path.exists('/content/bold-sorter-462413-e0-6cd62332c6d2.json'))  # Should print True


True


In [ ]:
ngrok.kill()